## Images

In [ ]:

# Build an image from DockerFile 
docker build -t <my_tag> .

docker images

docker images <image_name>

docker images --filter dangling=true

docker images --filter label=maintainer



## Containers

In [ ]:

## Run a container from an existing image
docker run -p machineport:containerport <image_name_or_image_id>

## Run a container and give a name

docker run <image_name_or_id> --name my_name

## show logs
sudo docker logs xxxxxx

## connect to the docker container shell
docker exec -it soglomania/portfolio bash
docker exec -it soglomania/portfolio sh

docker ps

docker ps --format {{.Names}}
docker ps --format {{.ID}}


## Cleaning

In [ ]:
# delete all images

sudo docker rmi $(sudo docker images -a -q)

#delete untaged images

sudo docker rmi $(sudo docker images -a | grep "^<none>" | awk '{print $3}')

sudo docker rmi $(sudo docker images -f "dangling=true" -q)


#stop containers and remove running containers

sudo docker rm -f  $(sudo docker ps -a -q )

## Docker Compose

In [ ]:

## docker command to start and stop

sudo docker-compose up
sudo docker-compose up -d #run in background

sudo docker-compose down
sudo docker-compose down --volumes #delete mounted files

# stop everything
sudo docker-compose stop

#run a command on a service in the docker-compose.yaml file
sudo docker-compose run service_name command 


## list services
sudo docker-compose ps

##when i want to rebuild the images
sudo docker-compose build

## rebuid after renaming service
sudo docker-compose up --remove-orphans

## exec command on a docker-compose service log

sudo docker-compose exec log tail -f /var/log/portfolio.log


## Build a PHP Application

In [ ]:
### Apache.conf

<VirtualHost *:80>
  ServerAdmin arcadius.soglo@orange.com
  DocumentRoot /var/www/gui

  <Directory /var/www/gui/>
      #Options Indexes FollowSymLinks MultiViews
      #AllowOverride All
      Order deny,allow
      Allow from all
  </Directory>

  Alias /packages /var/www/packages
  <Directory /var/www/packages>
    Order Allow,Deny
    Allow from all
  </Directory>

  ErrorLog ${APACHE_LOG_DIR}/error.log
  CustomLog ${APACHE_LOG_DIR}/access.log combined

</VirtualHost>

In [ ]:
## Dockerfile

FROM ubuntu:18.04

LABEL MAINTAINER Arcadius SOGLO <arcadius.soglo@orange.com>

RUN apt-get update && apt-get -y install software-properties-common dirmngr apt-transport-https lsb-release ca-certificates

# Install apache, PHP, and supplimentary programs. openssh-server, curl, and lynx-cur are for debugging the container.
RUN add-apt-repository ppa:ondrej/php && add-apt-repository ppa:ansible/ansible && apt-get update && apt-get -y upgrade && DEBIAN_FRONTEND=noninteractive apt-get -y install \
    apache2 php7.0 php7.0-mysql libapache2-mod-php7.0 curl iputils-ping ansible python-pip vim

# Install python dependencies
RUN pip install pexpect

# Enable apache mods.
RUN a2enmod php7.0
RUN a2enmod rewrite

# Update the PHP.ini file, enable <? ?> tags and quieten logging.
RUN sed -i "s/short_open_tag = Off/short_open_tag = On/" /etc/php/7.0/apache2/php.ini
RUN sed -i "s/error_reporting = .*$/error_reporting = E_ERROR | E_WARNING | E_PARSE/" /etc/php/7.0/apache2/php.ini


# Update /etc/ssh/ssh_config enable insecure ciphers
RUN sed -i "s/#.*.Ciphers.*/    Ciphers aes128-ctr,aes192-ctr,aes256-ctr,aes128-cbc,3des-cbc/" /etc/ssh/ssh_config

# Manually set up the apache environment variables
ENV APACHE_RUN_USER www-data
ENV APACHE_RUN_GROUP www-data
ENV APACHE_LOG_DIR /var/log/apache2
ENV APACHE_LOCK_DIR /var/lock/apache2
ENV APACHE_PID_FILE /var/run/apache2.pid

# Expose apache.
EXPOSE 80

# Create log dir
RUN mkdir /var/www/html/log/

# Copy web portal repo
ADD gui /var/www/gui

# Create packages dir
RUN mkdir /var/www/packages

# Copy ansible repo
ADD ansible /var/www/ansible

RUN chown -R www-data:www-data /var/www/

# Update the default apache site with the config we created.
ADD apache.conf /etc/apache2/sites-enabled/000-default.conf

# By default start up apache in the foreground, override with /bin/bash for interative.
CMD /usr/sbin/apache2ctl -D FOREGROUND

### Build a Flask Application

In [ ]:
## Dockerfile

FROM ubuntu:18.04

LABEL maintainer.fullname="SOGLO Arcadius"
LABEL maintainer.email="arcadius.soglo@orange.com" 


ENV WORKING_DIR /home/docker/code

ENV ANSIBLE_WORKING_DIR ${WORKING_DIR}/ansible
ENV ANSIBLE_LOGS_DIR ${WORKING_DIR}/logs
ENV GUI_WORKING_DIR ${WORKING_DIR}/gui
ENV PACKAGES_DIR ${WORKING_DIR}/packages

ENV NGINX_RUN_USER www-data
ENV NGINX_RUN_GROUP www-data

# Install required packages

RUN apt-get update && apt-get -y install software-properties-common dirmngr apt-transport-https lsb-release ca-certificates

RUN add-apt-repository ppa:ansible/ansible && \
	apt-get update && \
	apt-get -y upgrade && \
	DEBIAN_FRONTEND=noninteractive apt-get -y install \
    curl \
	git \
	build-essential \
	vim \
	iputils-ping \
	python3 \
	python3-dev \
	python3-setuptools \
	python3-pip \
	ansible \
	nginx \
	supervisor \	
	sqlite3 && \
	pip3 install -U pip setuptools && \
   	rm -rf /var/lib/apt/lists/*

RUN mkdir -p ${WORKING_DIR}/ \
	&& mkdir -p ${ANSIBLE_WORKING_DIR} && mkdir -p ${GUI_WORKING_DIR} \
	&& mkdir -p ${ANSIBLE_LOGS_DIR} && mkdir -p ${PACKAGES_DIR} \
	&& chown -R ${NGINX_RUN_USER}:${NGINX_RUN_GROUP} ${WORKING_DIR} \
	&& chmod 777 ${WORKING_DIR}/ \
	&& chmod 777 /run/ -R \
	&& chmod 777 /root/ -R

RUN pip3 install uwsgi

# COPY requirements.txt and RUN pip install BEFORE adding the rest of the code, 
# this will cause Docker's caching mechanism
# to prevent re-installing all dependencies if there is no change in requirements.txt
COPY requirements.txt ${WORKING_DIR}/
RUN pip3 install -r ${WORKING_DIR}/requirements.txt

# Settings file
COPY settings/nginx.conf /etc/nginx/nginx.conf
COPY settings/nginx-app.conf /etc/nginx/conf.d/
COPY settings/supervisor-app.conf /etc/supervisor/conf.d/

COPY settings/uwsgi_params ${WORKING_DIR}/uwsgi_params
COPY settings/uwsgi.ini ${WORKING_DIR}/uwsgi.ini

# add the gui code
COPY gui ${GUI_WORKING_DIR}

# add ansible code
COPY ansible ${ANSIBLE_WORKING_DIR}

# Change owner of ${WORKING_DIR}
RUN chown -R ${NGINX_RUN_USER}:${NGINX_RUN_GROUP} ${WORKING_DIR}/

EXPOSE 80

WORKDIR ${WORKING_DIR}

CMD ["supervisord", "-n"]

In [ ]:
## nginx-app.conf

upstream flask {
    server unix:/run/uwsgiApp.sock;
}


server {
  
    # the port your site will be served on, default_server indicates that this server block
    # is the block to use if no blocks match the server_name
    listen 80;

    # the domain name it will serve for
    server_name localhost;

    charset utf-8;


	location /robots.txt {
		alias /home/docker/code/gui/static/robots.txt;
    }

    location /humans.txt {
		alias /home/docker/code/gui/static/humans.txt;
    }

    location /static {
        alias /home/docker/code/gui/static;
    }

    location /packages {
        alias /home/docker/code/packages;
        autoindex on;
    }

    location /logs {
        alias /home/docker/code/logs;
        autoindex on;
    }
   
    location / {

        proxy_set_header Host $host;
        
        proxy_set_header X-Real-IP $remote_addr;

        include  /home/docker/code/uwsgi_params;

        uwsgi_pass flask;


    }
}


In [ ]:

## nginx.conf

user www-data;

worker_processes auto;

pid /run/nginx.pid;

events {
	worker_connections  20000;
}

http {
	##
        # Basic Settings
        ##

        sendfile on;
        tcp_nopush on;
        tcp_nodelay on;
        keepalive_timeout 65;
        types_hash_max_size 2048;
        server_tokens off;

        # server_names_hash_bucket_size 64;
        # server_name_in_redirect off;
	include /etc/nginx/mime.types;
        default_type application/octet-stream;

	##
        # Virtual Host Configs
        ##
	include /etc/nginx/conf.d/*;

	##
        # Logging Settings
        ##

        access_log /var/log/nginx/access.log;
        error_log /var/log/nginx/error.log;

        ##
        # Gzip Settings
        ##
	gzip off;
        #gzip_disable "msie6";
}

daemon off;

In [ ]:
## supervisor-app.conf

[supervisord]
nodaemon = true
user = root
#loglevel = info

[program:uwsgi]
command = /usr/local/bin/uwsgi --ini /home/docker/code/uwsgi.ini
#stdout_logfile=/dev/stdout
#stdout_logfile_maxbytes=0
#stderr_logfile=/dev/stderr
#stderr_logfile_maxbytes=0

[program:nginx]
command = /usr/sbin/nginx
#stdout_logfile=/dev/stdout
#stdout_logfile_maxbytes=0
#stderr_logfile=/dev/stderr
#stderr_logfile_maxbytes=0

In [ ]:
## wgsi.ini

[uwsgi]
# this config will be loaded if nothing specific is specified
# load base config from below
ini = :base

# %d is the dir this configuration file is in
socket = /run/uwsgiApp.sock
pidfile = /run/.pid
processes = 4
threads = 2

#set uid flag, get root access without running as root
uid = www-data
gid = www-data
master = true

[dev]
ini = :base
# socket (uwsgi) is not the same as http, nor http-socket
socket = :8001


[local]
ini = :base
http = :8000
# set the virtual env to use
home=/usr/bin/env


[base]
#plugins = /usr/lib/uwsgi/python3
# chdir to the folder of this config file, plus gui/
chdir = %dgui/
# load the module from wsgi.py, it is a python path from 
# the directory above.
module=gui:app

# allow anyone to connect to the socket. This is very permissive
# chmod-socket=666